In [14]:
!rm -f ru_tg_train.tar.gz
!wget https://www.dropbox.com/s/97cw9n001l4altq/ru_tg_train.tar.gz
!rm -f ru_tg_train.json
!tar -xzvf ru_tg_train.tar.gz
!rm ru_tg_train.tar.gz

--2021-07-26 11:15:30--  https://www.dropbox.com/s/97cw9n001l4altq/ru_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/97cw9n001l4altq/ru_tg_train.tar.gz [following]
--2021-07-26 11:15:31--  https://www.dropbox.com/s/raw/97cw9n001l4altq/ru_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc12d020c550a64a56de7d238636.dl.dropboxusercontent.com/cd/0/inline/BTDBoYdGaH_9UdyV1ejSrPKJ0txLVUEMmu-YCYNLLTgHLoi0y-C0XfGWjHz07yTccnhNvk8GVPJAj-W4HaNyYoguukNmLJE-YV0IH69pDXanG-CtEr2rVQoEw74f2RGLuGg8Bdm87jJ_5sPLxuXhv-6X/file# [following]
--2021-07-26 11:15:31--  https://uc12d020c550a64a56de7d238636.dl.dropboxusercontent.com/cd/0/inline/BTDBoYdGaH_9UdyV1ejSrPKJ0txLVUEMmu-YCYNLLTgHLoi0y-C0XfGWjHz07yTccnhNvk8G

In [15]:
!rm -f en_tg_train.tar.gz
!wget https://www.dropbox.com/s/oz0fcd4d5jejoog/en_tg_train.tar.gz
!rm -f en_tg_train.json
!tar -xzvf en_tg_train.tar.gz
!rm en_tg_train.tar.gz

--2021-07-26 11:15:37--  https://www.dropbox.com/s/oz0fcd4d5jejoog/en_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/oz0fcd4d5jejoog/en_tg_train.tar.gz [following]
--2021-07-26 11:15:37--  https://www.dropbox.com/s/raw/oz0fcd4d5jejoog/en_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50b24f3de9a83429230dc2b1d9.dl.dropboxusercontent.com/cd/0/inline/BTAUMDujGXPDXL3asuzIOm99J2eQ7Wgd9DPXtvfPRj1hxtBI3F4E6hQVSnT14KwJnjpJOVSR26rLDgTQj6xImJv4eFMaxwhXUKP6kFNX9cgPo8S3sGOaQmMzy_qOOu_blnCcVEOoT-mlBKeIjdWIXApj/file# [following]
--2021-07-26 11:15:37--  https://uc50b24f3de9a83429230dc2b1d9.dl.dropboxusercontent.com/cd/0/inline/BTAUMDujGXPDXL3asuzIOm99J2eQ7Wgd9DPXtvfPRj1hxtBI3F4E6hQVSnT14KwJnjpJOVSR

In [16]:
!pip install url-normalize

In [17]:
import sys
import json
from url_normalize import url_normalize
from urllib.parse import urlparse   


def normalize_url(url):
    url_n = url_normalize(url)
    parsed = urlparse(url_n)
    scheme = "%s://" % parsed.scheme
    res = parsed.geturl().replace(scheme, '', 1)
    host = parsed.netloc
    if res.startswith('www.'):
        res = res.replace('www.', '')
        host = host.replace('www.', '')
    return res, host

data_files = [json.load(open("en_tg_train.json")), json.load(open("ru_tg_train.json"))]

url_set = set()
host_set = set()
for data in data_files:
    for row in data:
        url = row["url"]
        url_n, url_host = normalize_url(url)
        url_set.add(url_n)
        host_set.add(url_host)

In [18]:
from collections import Counter, defaultdict

count = Counter()
host_to_host = defaultdict(Counter)
for data in data_files:
    for row in data:
        url = row["url"]
        url_n, url_host = normalize_url(url)
        if "out_links" not in row:
            continue
        for link in row["out_links"]:
            link_n, link_host = normalize_url(link)
            if url_host != link_host and link_n in url_set:
                count[link] += 1
            if url_host != link_host and link_host in host_set:
                host_to_host[url_host][link_host] += 1

h2i = {}
i2h = []
for host in host_to_host:
    h2i[host] = len(i2h)
    i2h.append(host)

In [19]:
import numpy as np

size = len(i2h)
E = np.ones((size, size))
E = E / np.sum(E, axis=1, keepdims=True)
print(E)

[[0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]
 [0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]
 [0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]
 ...
 [0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]
 [0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]
 [0.00271003 0.00271003 0.00271003 ... 0.00271003 0.00271003 0.00271003]]


In [20]:
matrix = np.zeros((size, size))

for host in host_to_host:
    for link_host in host_to_host[host]:
        if link_host not in h2i:
            continue
        matrix[h2i[host]][h2i[link_host]] = host_to_host[host][link_host]
print(matrix)

[[ 0.  0. 11. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [21.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [21]:
for i in range(size):
    if np.sum(matrix[i]) == 0:
        matrix[i] = np.ones(size)
print(matrix)

[[ 0.  0. 11. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [21.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [22]:
matrix = matrix / np.sum(matrix, axis=1, keepdims=True)
matrix = 0.7 * matrix + 0.3 * E
print(matrix)

[[0.00081301 0.00081301 0.01788618 ... 0.00081301 0.00081301 0.00081301]
 [0.00081301 0.00081301 0.00081301 ... 0.00081301 0.00081301 0.00081301]
 [0.04689451 0.00081301 0.00081301 ... 0.00081301 0.00081301 0.00081301]
 ...
 [0.00081301 0.00081301 0.00081301 ... 0.00081301 0.00081301 0.00081301]
 [0.00081301 0.00081301 0.00081301 ... 0.00081301 0.00081301 0.00081301]
 [0.00081301 0.00081301 0.00081301 ... 0.00081301 0.00081301 0.00081301]]


In [23]:
x = np.ones((1, size))
x = x / np.sum(x, axis=1, keepdims=True)
print(x)

[[0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.00271003 0.00271003
  0.00271003 0.00271003 0.00271003 0.00271003 0.

In [24]:
y = np.dot(x, np.linalg.matrix_power(matrix, 20))
print(y)

[[0.00170784 0.00208693 0.00331435 0.00100501 0.00085165 0.00296359
  0.00290969 0.00094703 0.00214449 0.00085165 0.00189471 0.00085165
  0.00085165 0.00877118 0.0014871  0.00132941 0.05106091 0.00149211
  0.00085165 0.0144967  0.00085165 0.00320484 0.00085165 0.00085165
  0.13518963 0.01168638 0.00099793 0.00099113 0.00133939 0.00469615
  0.00158749 0.00092617 0.00333858 0.00303675 0.00110138 0.00085165
  0.00085572 0.00091879 0.00203584 0.00093165 0.00085165 0.00085165
  0.0027937  0.00795758 0.00085165 0.00086894 0.0090068  0.0127743
  0.00213523 0.00153604 0.00085165 0.00090754 0.00087544 0.01187057
  0.00334402 0.00099102 0.00085165 0.00085812 0.0009511  0.00206571
  0.00182254 0.00115682 0.00085165 0.00085165 0.03858815 0.00085165
  0.00957244 0.00086849 0.00891725 0.00568722 0.00085165 0.00092453
  0.00140135 0.00087028 0.00085165 0.00139193 0.00095472 0.00085165
  0.00089361 0.00087028 0.00085165 0.00085165 0.00085165 0.00294083
  0.00115988 0.00101947 0.00095707 0.00093681 0.0

In [25]:
rating = []
for i, host in enumerate(i2h):
    rating.append((i, y[0][i], host))
rating = sorted(rating, key=lambda x:-x[1])
for i, pr, host in rating:
    print(pr, host)

0.1351896332545013 ria.ru
0.05106091295103805 kommersant.ru
0.04029658853961788 iz.ru
0.03858814844619006 interfax.ru
0.03851889910079499 rbc.ru
0.03801329290158776 russian.rt.com
0.02110448958273943 rg.ru
0.020401326050801662 360tv.ru
0.014496697810145313 ren.tv
0.013279206242478126 mskagency.ru
0.012774299833140847 eurointegration.com.ua
0.011870572093560496 epravda.com.ua
0.011686383755329283 pravda.com.ua
0.010706348379275737 nation-news.ru
0.010510492903098304 ura.news
0.009841007806573175 5-tv.ru
0.009572438500552958 rsport.ria.ru
0.009006796959293392 fontanka.ru
0.008917250349610658 lenta.ru
0.008771176674908721 nv.ua
0.007957583424039568 meduza.io
0.007791069800817555 nplus1.ru
0.007400986689131512 tvzvezda.ru
0.007345724631450593 vesti.ru
0.005979671201805142 sport-interfax.ru
0.00568722300167357 novayagazeta.ru
0.005213998101191683 radiosputnik.ria.ru
0.005174476338340479 rueconomics.ru
0.004820114552610658 obozrevatel.com
0.004696145200925906 zona.media
0.004625617526780021 

In [ ]:
0.05511665994787871 nytimes.com
0.04232894744295712 bloomberg.com
0.037647787907772755 forbes.com
0.03614312420561435 ria.ru
0.03529431498509583 theguardian.com
0.023043659191782036 reuters.com
0.02092053045639703 mirror.co.uk
0.020758953442467056 bbc.co.uk
0.013871023570674577 kommersant.ru
0.013866615406266254 theverge.com
0.013581232140313225 thesun.co.uk
0.01106746029435335 iz.ru
0.010925324372022406 hollywoodreporter.com
0.010508062691059025 rbc.ru
0.010459412364063347 interfax.ru
0.009666022013528227 russian.rt.com
0.009535732547369119 politico.com
0.00926787785043659 ft.com
0.009175715590080662 telegraph.co.uk
0.0076832763506289635 independent.co.uk
0.007606665179379701 businessinsider.com
0.007462148583606343 nature.com
0.006900816741165872 nypost.com
0.006758676920675725 techcrunch.com
0.006736039250769494 rg.ru
0.00579443048312328 theconversation.com
0.005729029336235546 cbsnews.com
0.0054240465107435925 360tv.ru
0.005181448284523417 foxnews.com
0.004645394774737957 wired.com
0.004190179235055331 theatlantic.com
0.0038374562709204514 ren.tv
0.003751503876358776 nation-news.ru
0.0037465890191596884 metro.co.uk
0.0037320314626537606 mskagency.ru
0.0036815205344362476 abcnews.go.com
0.003663303663893805 thehill.com
0.0035750725432424947 polygon.com
0.003459595180666024 ura.news
0.003359729871887461 gizchina.com
0.003347721746569902 abc.net.au
0.0033169881948614216 lenta.ru
0.00327109087839729 skysports.com
0.003236560465399384 eurointegration.com.ua
0.003225193432955941 time.com
0.003146223094002235 pravda.com.ua
0.0031039267097927566 newyorker.com
0.002943893739506842 meduza.io
0.002735312263678417 5-tv.ru
0.002725049114870761 fontanka.ru
0.002666466730493621 rsport.ria.ru
0.0025051088041102045 zdnet.com
0.002449430553090048 xda-developers.com
0.002431832019806434 eurogamer.net
0.002405146679652312 epravda.com.ua
0.002314012671299186 androidpolice.com
0.0022908204047905117 aljazeera.com
0.002202131643392858 marketwatch.com
0.0021869001402905317 nv.ua
0.002184247516888205 engadget.com
0.00214334979499565 9to5mac.com
0.002030754970496375 vg247.com
0.0020062782902026677 gsmarena.com
0.0020028407158378175 vesti.ru
0.0020007777909044045 harpersbazaar.com
0.001998766529472067 dailyrecord.co.uk
0.0019650039792819107 tvzvezda.ru
0.0019582372846873477 propublica.org
0.0019255214456248488 liverpoolecho.co.uk
0.0018550174543300341 novayagazeta.ru
0.0018504889603722823 abc7.com
0.0018453962997608082 ctvnews.ca
0.0018431770403709695 markets.businessinsider.com
0.001832543145608844 seekingalpha.com
0.001761337333246139 techradar.com
0.001759970742314549 radiosputnik.ria.ru
0.0016671696483547209 venturebeat.com
0.0016615552214634683 abc7ny.com
0.001654876902121346 zona.media
0.0016177554122857127 elle.com
0.0015995232765197381 rueconomics.ru
0.0015936050297023 indianexpress.com
0.0015879894878407708 nplus1.ru
0.0015804867309747368 pnp.ru
0.0015732254722512534 haaretz.com
0.0015716845859265333 9to5google.com
0.0015564412458324984 hbr.org
0.0015539324569728385 theintercept.com
0.0015422136727038724 pcgamesn.com
0.0015402074351041773 znak.com
0.001509892215765582 washingtonexaminer.com
0.001509563480999023 arstechnica.com
0.0014885919457464749 fivethirtyeight.com
0.0014792582698298893 mashable.com
0.001465920480179618 obozrevatel.com
0.0014554226493632126 quantamagazine.org
0.0014179902896359524 nme.com
0.0013776493331242718 sammobile.com
0.0013608492992956153 motor1.com
0.0013581122904712509 vulture.com
0.0013360478052293861 walesonline.co.uk
0.001322091896045477 nationalinterest.org
0.0013219006652169922 hongkongfp.com
0.0012904238530937714 pcgamer.com
0.001277338452879835 sport-interfax.ru
0.0012745633116026092 genius.com
0.0012391227408802786 irishmirror.ie
0.0012295547109018533 macrumors.com
0.0012276611862861695 korrespondent.net
0.0012275058443061064 chroniclelive.co.uk
0.0012157936695614672 eurekalert.org
0.0012076692843816877 androidcentral.com
0.001204429868873306 nur.kz
0.0011974173230379867 superomsk.ru
0.0011947192086737188 sportsdaily.ru
0.0011919905692971695 nevnov.ru
0.001186018886706588 thenextweb.com
0.001173920097491342 sputniknews.com
0.0011618402370716934 archdaily.com
0.0011596336349215311 newizv.ru
0.0011555176661665993 7days.ru
0.0011369285483799352 dni.ru
0.0011310863298095916 golos-ameriki.ru
0.0011305059313958883 thewrap.com
0.0011167555708688924 interfax-russia.ru
0.001116012933714049 cinemablend.com
0.0011101512498848062 birminghammail.co.uk
0.0011049995753765352 starhit.ru
0.001103274301009521 tvrain.ru
0.0010675426919076751 newsone.ua
0.0010563791346247507 coindesk.com
0.0010526284521901186 life.ru
0.0010517151290297313 zn.ua
0.0010484938298680462 realty.ria.ru
0.0010436974645263684 seattletimes.com
0.0010359809936262665 cosmopolitan.com
0.001034940251347824 punchng.com
0.0010326879868254063 glavcom.ua
0.001032276934416259 windowscentral.com
0.0010321504705361041 ru.krymr.com
0.0010112519236404255 soyacincau.com
0.0010052073494330905 pitchfork.com
0.0009981149806242457 scientificamerican.com
0.0009904362251983633 ukranews.com
0.00098110053198324 channelnewsasia.com
0.0009708879327426497 thehindu.com
0.0009702438443513347 mentalfloss.com
0.000965631834338584 malaymail.com
0.0009649599073666223 espn.co.uk
0.000960275032072626 inforeactor.ru
0.00095493218068878 1prime.ru
0.000952123029476517 news.sportbox.ru
0.0009402241837997966 rosbalt.ru
0.000937239823370145 politifact.com
0.0009301149473409567 pobeda26.ru
0.0009296286249898584 bryansktoday.ru
0.0009289616086976662 refinery29.com
0.0009241148241333444 japantimes.co.jp
0.0009223482751938444 searchengineland.com
0.0009118645528676392 politico.eu
0.0009100414314344824 inosmi.ru
0.000902362694943051 hypebeast.com
0.0008967497557807696 androidauthority.com
0.0008907569799146019 nintendolife.com
0.0008818583965607504 tjournal.ru
0.0008780290110108801 scotsman.com
0.0008770082445353059 sportsnet.ca
0.0008730065866816661 smithsonianmag.com
0.0008620977332033605 news-r.ru
0.0008617016595804199 theglobeandmail.com
0.0008597977871063051 thejakartapost.com
0.0008573414739686625 destructoid.com
0.0008566032101415455 appleinsider.com
0.0008545026073826155 newstatesman.com
0.0008446860450316404 bristolpost.co.uk
0.0008327424951970114 sputnik.by
0.0008320974753441367 theweek.com
0.0008279841848501546 slovodel.com
0.0008269842467866546 gordonua.com
0.0008268212102298424 mspoweruser.com
0.0008220271196229674 news.yahoo.com
0.0008213720785310039 sportingnews.com
0.0008212852255547034 rns.online
0.0008207550310753637 xxlmag.com
0.0008172769754140413 dsogaming.com
0.0008129591071120953 newsvl.ru
0.0008085378461525263 gorod55.ru
0.0007913502605282545 sfgate.com
0.0007857259814384939 thebell.io
0.0007805408061636277 sciencealert.com
0.0007799830993190511 marieclaire.com
0.0007734716770533694 moscow-baku.ru
0.0007658065533356675 bmwblog.com
0.0007636298693892212 medikforum.ru
0.0007616479110651787 scoop.co.nz
0.0007600145343723778 pravda.ru
0.0007514783896867316 wccftech.com
0.0007466077088597293 mmamania.com
0.0007459185734246681 trueachievements.com
0.0007441528779434975 versiya.info
0.0007440260240993779 dailytelegraph.com.au
0.0007437010043043119 interestingengineering.com
0.0007416846284610568 politexpert.net
0.0007326731070668257 eff.org
0.0007270621763031494 theage.com.au
0.0007249672505750169 inc.com
0.000719218344073455 fedpress.ru
0.0007189564068726637 cbr.com
0.0007186710919238799 slashfilm.com
0.0007182903946105908 ru.rfi.fr
0.0007177023976872566 apostrophe.ua
0.0007167212167680909 autoblog.com
0.0007164671675669029 sport.business-gazeta.ru
0.0007164165036301985 the-village.ru
0.000712425925984605 leaderpost.com
0.0007116763394951199 nottinghampost.com
0.0007021222478824652 thestarphoenix.com
0.0007020053348297118 bloodyelbow.com
0.0006965274182873318 wpxi.com
0.0006951966403591088 rzn.info
0.0006940602961827166 sherdog.com
0.0006934825553441055 livelaw.in
0.0006905976263859 ru.euronews.com
0.0006898557458292509 spbdnevnik.ru
0.0006898426793579509 tengrinews.kz
0.0006842897495586028 thisisanfield.com
0.0006826451760578921 takiedela.ru
0.0006740664035515659 pushsquare.com
0.0006720329343314656 allhiphop.com
0.0006717823477377686 architects.d3.ru
0.0006702399430294393 euronews.com
0.0006698839911767544 ovdinfo.org
0.000665991415944459 newsru.com
0.000665839355458039 c-inform.info
0.0006622020625098046 futurism.com
0.0006611208974887058 colta.ru
0.0006608145033327287 lifetambov.ru
0.0006595919736113286 atlasobscura.com
0.0006563094678915267 premiumtimesng.com
0.0006560918049925066 vladtime.ru
0.0006552062753629968 vpravda.ru
0.0006548179543109159 newrepublic.com
0.0006541070295496595 standardmedia.co.ke
0.0006541070295496595 zr.ru
0.0006541070295496583 allkpop.com
0.0006541070295496574 mybroadband.co.za
0.0006525297991859454 newinform.com
0.0006518400721395379 imore.com
0.0006419996266545597 svpressa.ru
0.0006413819896843025 washingtontimes.com
0.0006403042825675009 trustedreviews.com
0.0006359157328728364 geek.com
0.0006340097143228437 dsnews.ua
0.0006286472209513644 ridus.ru
0.0006250092035853837 pymnts.com
0.0006246644126948405 teamtalk.com
0.0006244806458888874 eg.ru
0.0006241769938139403 courthousenews.com
0.0006232199877467347 businessinsider.com.au
0.00061961083544533 nationalpost.com
0.0006153434977440945 devonlive.com
0.000613587734676966 hiphopdx.com
0.0006131265328320789 androidheadlines.com
0.0006128209454421648 computerworld.com
0.0006123775262647635 myslo.ru
0.0006095042761037874 football.london
0.0006074282550590179 factcheck.org
0.0006072656057160051 vladnews.ru
0.000603655145924039 onlinetambov.ru
0.0006032551385911152 sputnik-georgia.ru
0.0006021883640958116 cointelegraph.com
0.0006015576146754732 bleepingcomputer.com
0.0006013418675819364 thefashionlaw.com
0.0005995658130562663 vz.ru
0.0005965322740687618 7x7-journal.ru
0.0005953921802806289 spin.com
0.0005948991655527054 sport24.co.za
0.0005917722738285706 vc.ru
0.0005899029860591414 thescottishsun.co.uk
0.000587794466515409 techspot.com
0.0005876462012331082 breitbart.com
0.0005844772017817256 bgr.com
0.000581504409671896 derbytelegraph.co.uk
0.000580787876705363 dp.ru
0.0005792869471441372 psychologytoday.com
0.0005782880903001761 talkingpointsmemo.com
0.0005743756452113943 gizmochina.com
0.0005710351076515575 vesma.today
0.0005694516257481607 upi.com
0.0005685440339470165 dawn.com
0.000565426600284812 inforesist.org
0.0005644679141181379 shapito.d3.ru
0.0005643276338684866 foxsportsasia.com
0.0005639449963419854 mlb.com
0.0005626256753317995 tech.slashdot.org
0.0005622814402288075 news.rambler.ru
0.000559491576896421 plymouthherald.co.uk
0.0005552866356688292 kotaku.com.au
0.0005526120540203128 komiinform.ru
0.0005514498590281097 toronto.ctvnews.ca
0.0005489327438190398 politros.com
0.0005483880501297717 newstracker.ru
0.000547453854518938 glavny.tv
0.0005465497349451987 techdirt.com
0.0005451281581021559 techno.nv.ua
0.0005431436271221604 inopressa.ru
0.0005405786867485851 hothardware.com
0.0005382412339597395 ria56.ru
0.0005377815766775735 metronews.ru
0.0005358081356762062 planet-today.ru
0.0005353650759194601 riamo.ru
0.0005300631832312828 business-standard.com
0.0005293132122166945 boston25news.com
0.0005276329421066145 loudersound.com
0.0005261076667756807 gazeta.uz
0.0005242687177168244 theinquirer.net
0.000524244375364004 sib.fm
0.0005236019329681269 theins.ru
0.000522983759240454 hsj.co.uk
0.0005226903236223398 utro.ru
0.0005223838223925868 pasmi.ru
0.0005216150316661896 realnoevremya.ru
0.0005214370391809328 gamasutra.com
0.0005212076399441118 gizmodo.com.au
0.0005192511191098466 pravo.ru
0.0005189894611439964 m24.ru
0.000518526298574006 calgaryherald.com
0.000514790187049025 artnews.com
0.0005144305731473841 sakhalinmedia.ru
0.0005112981592895429 yuga.ru
0.000510772286476168 bloombergquint.com
0.0005107193895379338 business.financialpost.com
0.0005106776838929808 shacknews.com
0.0005097796048976179 bc.ctvnews.ca
0.0005058092609857643 bitcoinist.com
0.0005032395654982003 calendar.fontanka.ru
0.0005015804843400708 inkazan.ru
0.0004989098256610532 penza-press.ru
0.0004983927971033174 golos.ua
0.0004915684194257228 bustle.com
0.0004912126911140458 tribune.com.pk
0.00048719682636819784 memepedia.ru
0.00048660926829023664 infox.ru
0.0004861323224095855 moneycontrol.com
0.00048481351122252526 gizmodo.co.uk
0.0004843696382521011 cyberscoop.com
0.0004832002740692893 unilad.co.uk
0.00048201514048707404 magadanmedia.ru
0.00048201514048707404 krasnoyarskmedia.ru
0.0004806403337853233 komonews.com
0.00048002827021352367 carbuzz.com
0.00048002827021352367 caradvice.com.au
0.0004799134290397934 zakon.kz
0.00047885565001216045 bloody-disgusting.com
0.0004784845622878777 aeon.co
0.00047768732607276804 nylon.com
0.00047735384430235223 politota.d3.ru
0.00047683769218749626 daryo.uz
0.0004758365262767553 paperpaper.ru
0.0004745482381873156 ulpressa.ru
0.00047383595898754034 abcactionnews.com
0.0004731902090984244 thenewdaily.com.au
0.0004725750586815098 calgary.ctvnews.ca
0.0004716373941759884 liliputing.com
0.0004701780023633998 t3.com
0.0004699655471598839 news24.com
0.00046863330574355616 interest.co.nz
0.0004670289709082965 news.slashdot.org
0.0004663614317317796 moika78.ru
0.00046545501965123727 freemedia.io
0.00046499784563827764 actualnews.org
0.0004642319648842516 foxsports.com
0.0004642148497905567 riftherald.com
0.00046308048473562007 vedtver.ru
0.00046172346881666126 montreal.ctvnews.ca
0.00046062534472140267 news-front.info
0.0004593607518215588 thesixthaxis.com
0.00045621078137651587 nokiamob.net
0.00045615380585579246 cityam.com
0.00045498783502282846 asn24.ru
0.00045462880754300507 btu.org.ua
0.00045410342781923634 bk55.ru
0.00045289744526677393 vzsar.ru
0.00045169608440306507 boingboing.net
0.0004509002639760267 newsbtc.com
0.0004508809313656662 itsmycity.ru
0.0004495997078267635 riatomsk.ru
0.0004495997078267635 susanin.news
0.0004495997078267635 irkutskmedia.ru
0.0004490926587803118 iflscience.com
0.0004477371646733542 polka.d3.ru
0.00044752011527071336 rferl.org
0.00044723400150504974 mos.news
0.0004465147643269889 mobile.slashdot.org
0.0004437156449910485 financemagnates.com
0.0004420837375852302 yro.slashdot.org
0.00044184724591728896 slashdot.org
0.00044162246031392946 pastemagazine.com
0.00044151089653326304 nakanune.ru
0.0004397357257281834 newstula.ru
0.00043918733033822564 infpol.ru
0.00043776000266850727 forpost-sz.ru
0.00043732100785772967 betanews.com
0.00043655247899515375 apple.slashdot.org
0.00043406650928453906 blockonomi.com
0.00043286354973956617 brooklynvegan.com
0.0004322871755768262 ccn.com
0.00043117691397683174 vestiprim.ru
0.0004310441702610151 raspberrypi.org
0.00043054589990538324 uaprom.info
0.00043052373666201154 news.ykt.ru
0.0004300168421121004 xakep.ru
0.0004296989312282062 dezeen.com
0.00042941728797444515 stereogum.com
0.0004294014378363924 kotaku.co.uk
0.00042900292926900056 politics.slashdot.org
0.00042866620922056884 snob.ru
0.00042830554999096973 omskzdes.ru
0.00042790922142509107 ysia.ru
0.00042726692753337176 yugopolis.ru
0.00042724428307335335 irishcentral.com
0.00042707357754755425 popmech.ru
0.00042609321397417475 perthnow.com.au
0.0004258986035762001 huffingtonpost.in
0.0004258531136041861 style.nv.ua
0.00042581475561459937 ria-m.tv
0.00042581475561459937 forklog.com
0.00042526132933297476 science.slashdot.org
0.00042524399926805557 politics.nur.kz
0.00042501169928601645 unian.info
0.00042411288401889306 appleinsider.ru
0.0004234963605542791 openaccessgovernment.org
0.00042239158282272307 motor.ru
0.00042202854257412324 hackaday.com
0.00042126256187720914 news.mail.ru
0.0004210931644692717 incrussia.ru
0.0004208694336757658 greenbiz.com
0.0004208694336757658 martechtoday.com
0.0004208694336757658 sustainablebrands.com
0.00042041711669571166 lawandcrime.com
0.00042007113513629625 mynintendonews.com
0.0004197607801851696 vokrug.tv
0.0004193225716897209 businessinsider.my
0.0004193225716897209 thespinoff.co.nz
0.0004193225716897209 balls.ie
0.0004193225716897209 nehandaradio.com
0.0004192440836877357 bleedingcool.com
0.0004191884882690165 eadaily.com
0.00041900570132409756 kerrang.com
0.00041888807271153093 dtf.ru
0.0004184361158865831 likeni.ru
0.00041843611588658305 iphones.ru
0.0004184361158865828 thesportreview.com
0.0004184361158865828 fotka.d3.ru
0.00041820931833433736 zik.ua
0.0004179604669353715 ultimateclassicrock.com
0.0004176961370679754 arbuztoday.ru
0.0004176961370679754 politics.d3.ru
0.00041762530058818843 ua.news
0.00041718427516645356 autoevolution.com
0.00041718427516645356 vest-news.ru
0.00041531580581270663 fakty.com.ua
0.0004134867783254172 runnersworld.com
0.0004133043114245777 oblgazeta.ru
0.00041309723770936984 bnkomi.ru
0.00041309723770936984 ptzgovorit.ru
0.000413083882682513 avtonovostidnya.ru
0.0004121684559128567 developers.slashdot.org
0.0004121684559128567 linux.slashdot.org
0.00041200687042700936 novorosinform.org
0.0004116369195958596 iharare.com
0.00041148919480245505 channel24.co.za
0.0004112061567113287 theedgemarkets.com
0.00041012709289137605 adindex.ru
0.00041001929754147265 img.d3.ru
0.0004092321886349503 nord-news.ru
0.0004084283401346369 itnews.com.au
0.00040824500074688655 2000.ua
0.00040740998881075015 hardware.slashdot.org
0.00040732220844345195 lifehacker.com.au
0.0004058019026662952 theindependent.sg
0.00040579803545650207 investorplace.com
0.0004055915724343561 uk.pcmag.com
0.0004040072844378229 punditarena.com
0.000403974489407639 discover24.ru
0.00040393673451069785 3dnews.ru
0.0004037635320795182 capital.ua
0.00040365914335804403 wonderzine.com
0.00040334723471064184 saratov24.tv
0.00040327889565335435 medialeaks.ru
0.00040308327082519435 gubdaily.ru
0.0004027508621059167 bfm.ru
0.0004027247216423774 caughtoffside.com
0.0004019962088286895 sport.ua
0.00040187466747682824 kiev.vgorode.ua
0.00040183570395005674 telegraf.com.ua
0.0004013744450424337 kanobu.ru
0.00040125627057755126 tmonews.com
0.00040076804547902633 sitv.ru
0.0004007420822009258 sci-news.com
0.000400408971833752 it.slashdot.org
0.00040022069443517577 severpost.ru
0.00039893487084855265 panram.ru
0.00039810830400170135 asi.org.ru
0.0003979326535155163 newsroom24.ru
0.00039748369606386 businessinsider.sg
0.0003974049787717018 criticalhit.net
0.0003970833471964248 bombardir.ru
0.000396195880894086 etcanada.com
0.00039610874214476995 popsugar.com
0.00039610286821250087 klops.ru
0.0003958552987967461 ain.ua
0.00039583594608621944 sports.ndtv.com
0.00039582833057414096 spaceflightnow.com
0.0003951301332013527 mamamia.com.au
0.00039408354548681895 from-ua.com
0.00039347447797014 trud.ru
0.00039322138062294876 almasdarnews.com
0.0003931856608512534 moscow.d3.ru
0.0003931856608512534 alles.d3.ru
0.0003931856608512534 historyporn.d3.ru
0.0003931856608512534 porebrik.d3.ru
0.00039288635025537804 mobilesyrup.com
0.00039274115716267073 fox4now.com
0.00039248692341283794 caoinform.ru
0.00039248692341283794 elementy.ru
0.0003923015376827566 business-gazeta.ru
0.00039226964244446383 tech.onliner.by
0.00039147631628921235 newdaynews.ru
0.000391438573253922 kod.ru
0.000391438573253922 gazeta19.ru
0.00039130724801997514 entertainment.slashdot.org
0.00039130403613989184 racurs.ua
0.0003911578997432056 argumentiru.com
0.0003911578997432056 regions.ru
0.0003911141462204188 editorial.rottentomatoes.com
0.0003906139864777368 coconuts.co
0.00039017309655117684 calcalistech.com
0.0003901207410786053 kianews24.ru
0.0003901207410786053 privet-rostov.ru
0.00039009968383577927 mig.com.ua
0.00038963140747994956 karpovka.com
0.00038951751489948734 for-ua.com
0.00038951751489948734 timeua.info
0.00038923455042052667 pg21.ru
0.000389149344547687 if24.ru
0.000389149344547687 dorinfo.ru
0.00038880341222130067 reporter-ua.com
0.00038870348028298966 runews24.ru
0.0003886055688173044 auto.onliner.by
0.00038840854773646125 vesti-sochi.tv
0.0003883127660198736 inquisitr.com
0.0003880955075078988 facenews.ua
0.000387995589131331 theoutline.com
0.0003878445793584408 vringe.com
0.0003878445793584408 igromania.ru
0.0003878399945743255 tproger.ru
0.0003878382235550725 kasparov.ru
0.0003876042344696533 vistanews.ru
0.00038734415116098176 vnnews.ru
0.0003872751728487501 physicsworld.com
0.0003872751728487501 techxplore.com
0.00038677882897661733 graniru.org
0.00038677882897661733 donpress.com
0.00038660732842452924 mel.fm
0.00038586808831948024 niann.ru
0.00038568808546533644 info24.ru
0.00038553618776837546 mogilev.online
0.0003853385405486422 askmen.com
0.00038503785569937753 mnn.com
0.00038476884250614445 oilcapital.ru
0.00038476884250614445 newsbuzz.ru
0.00038476884250614445 orda_mordora.d3.ru
0.00038476884250614326 junkee.com
0.00038476884250614326 theonlinecitizen.com
0.00038476884250614326 nairametrics.com
0.00038476884250614326 androidcommunity.com
0.00038476884250614326 channel3000.com
0.00038476884250614326 coinspeaker.com
0.00038476884250614326 blogs.tribune.com.pk
0.00038476884250614326 en.rfi.fr
0.00038476884250614326 blendernation.com
0.00038476884250614326 brightside.me
0.00038476884250614326 gpblog.com
0.00038476884250614326 everythingbarca.com
0.00038476884250614326 techweez.com
0.00038476884250614326 cryptopotato.com
0.00038476884250614326 thesportsdaily.com
0.00038476884250614326 hotnewhiphop.com
0.00038476884250614326 jantakareporter.com
0.00038476884250614326 breakingnews.ie
0.00038476884250614326 kahawatungu.com
0.00038476884250614326 atlantablackstar.com
0.00038476884250614326 businessmirror.com.ph
0.00038476884250614326 eteknix.com
0.00038476884250614326 idle.slashdot.org
0.00038476884250614326 sportscafe.in
0.00038476884250614326 rugbypass.com
0.00038476884250614326 omgubuntu.co.uk
0.00038476884250614326 superherohype.com
0.00038476884250614326 btcmanager.com
0.00038476884250614326 stratechery.com
0.00038476884250614326 thezimbabwemail.com
0.00038476884250614326 journal.com.ph
0.00038476884250614326 blog.avira.com
0.00038476884250614326 sohh.com
0.00038476884250614326 thespun.com
0.00038476884250614326 entertainmentdaily.co.uk
0.00038476884250614326 digitaljournal.com
0.00038476884250614326 infowars.com
0.00038476884250614326 empireofthekop.com
0.00038476884250614326 tbrfootball.com
0.00038476884250614326 icirnigeria.org
0.00038476884250614326 technave.com
0.00038476884250614326 face2faceafrica.com
0.00038476884250614326 thecourier.com.au
0.00038476884250614326 thehardtackle.com
0.00038476884250614326 ausdroid.net
0.00038476884250614326 ktvo.com
0.00038476884250614326 themeditelegraph.com
0.00038476884250614326 iafrica.com
0.00038476884250614326 heraldnet.com
0.00038476884250614326 mindbodygreen.com
0.00038476884250614326 pensionplanpuppets.com
0.00038476884250614326 moderndiplomacy.eu
0.00038476884250614326 crowdfundinsider.com
0.00038476884250614326 vladtv.com
0.00038476884250614326 macobserver.com
0.00038476884250614326 overdrive.in
0.00038476884250614326 designtaxi.com
0.00038476884250614326 games.slashdot.org
0.00038476884250614326 theprideoflondon.com
0.00038476884250614326 totalprosports.com
0.00038476884250614326 breakingisraelnews.com
0.00038476884250614326 dailyhodl.com
0.00038476884250614326 thecerbatgem.com
0.00038476884250614326 businesstech.co.za
0.00038476884250614326 justjaredjr.com
0.00038476884250614326 macrobusiness.com.au
0.00038476884250614326 gossipcop.com
0.00038476884250614326 worldofbuzz.com
0.00038476884250614326 kitchenertoday.com
0.00038476884250614326 technologynetworks.com
0.00038476884250614326 bangaloremirror.indiatimes.com
0.00038476884250614326 reddevilarmada.com
0.00038476884250614326 business.mb.com.ph
0.00038476884250614326 fiveaa.com.au
0.00038476884250614326 dunyanews.tv
0.00038476884250614326 elevenmyanmar.com
0.00038476884250614326 talkandroid.com
0.00038476884250614326 timescolonist.com
0.00038476884250614326 cbronline.com
0.00038476884250614326 bnn-news.com
0.00038476884250614326 tech.eu
0.00038476884250614326 designweek.co.uk
0.00038476884250614326 wsvn.com
0.00038476884250614326 newstalk.com
0.00038476884250614326 memeburn.com
0.00038476884250614326 paininthearsenal.com
0.00038476884250614326 iclarified.com
0.00038476884250614326 realmoney.thestreet.com
0.00038476884250614326 autoindustriya.com
0.00038476884250614326 anothermag.com
0.00038476884250614326 poundsterlinglive.com
0.00038476884250614326 thesector.com.au
0.00038476884250614326 wellington.scoop.co.nz
0.00038476884250614326 irishpost.com
0.00038476884250614326 artforum.com
0.00038476884250614326 channelnews.com.au
0.00038476884250614326 itnewsafrica.com
0.00038476884250614326 awardsdaily.com
0.00038476884250614326 foodnavigator.com
0.00038476884250614326 wwdmag.com
0.00038476884250614326 broadsheet.com.au
0.00038476884250614326 says.com
0.00038476884250614326 electronicsweekly.com
0.00038476884250614326 geektyrant.com
0.00038476884250614326 talkinginfluence.com
0.00038476884250614326 dailygalaxy.com
0.00038476884250614326 traveller.com.au
0.00038476884250614326 jewishpress.com
0.00038476884250614326 recipes.howstuffworks.com
0.00038476884250614326 css-tricks.com
0.00038476884250614326 mid-day.com
0.00038476884250614326 fashionweekdaily.com
0.00038476884250614326 parkinsonsnewstoday.com
0.00038476884250614326 cosmo.ru
0.00038476884250614326 charter97.org
0.00038476884250614326 timer-odessa.net
0.00038476884250614326 championat.asia
0.00038476884250614326 molnet.ru
0.00038476884250614326 glavnoe.ua
0.00038476884250614326 omskpress.ru
0.00038476884250614326 dev.by
0.00038476884250614326 sputnik-abkhazia.ru
0.00038476884250614326 ostro.org
0.00038476884250614326 kontrakty.ua
0.00038476884250614326 allhockey.ru
0.00038476884250614326 vlast.kz
0.00038476884250614326 topdialog.ru
0.00038476884250614326 35media.ru
0.00038476884250614326 experov.d3.ru
0.00038476884250614326 drugoigorod.ru
0.00038476884250614326 bigasia.ru
0.00038476884250614326 mvestnik.ru
0.00038476884250614326 sky.d3.ru
0.00038476884250614326 theguardian.d3.ru
0.00038476884250614326 chitaitext.ru
0.00038476884250614326 lifehacker.ru
0.00038476884250614326 cinemaholics.ru
0.00038476884250614326 mirf.ru
0.00038476884250614326 grodnonews.by
0.00038476884250614326 sibdepo.ru
0.00038476884250614326 trc33.ru
0.00038476884250614326 it.d3.ru
0.00038476884250614326 yesasia.ru
0.00038476884250614326 roem.ru
0.00038476884250614326 klerk.ru
0.00038476884250614326 rspectr.com
0.00038476884250614326 adme.ru
0.00038476884250614326 computerworld.ru
0.00038476884250614326 glupoff.d3.ru
0.00038476884250614326 anti-malware.ru
0.00038476884250614326 unionnews.ru
0.00038476884250614326 vashgorod.ru
0.00038476884250614326 aerospace.d3.ru
0.00038476884250614326 novoshishminsk.ru
0.00038476884250614326 vkonline.ru
0.00038476884250614326 nbnews.com.ua
0.00038476884250614326 mskgazeta.ru
0.00038476884250614326 top68.ru
0.00038476884250614326 nikvesti.com
0.00038476884250614326 stadium.ru
0.00038476884250614326 marimedia.ru
0.00038476884250614326 4pda.ru
0.00038476884250614326 primorye24.ru
0.00038476884250614326 agenda-u.org
0.00038476884250614326 gametech.ru
0.00038476884250614326 kineshemec.ru
0.00038476884250614326 shazoo.ru
0.00038476884250614326 real-vin.com
0.00038476884250614326 sport.rzn.info
0.00038476884250614326 lezgigazet.ru
0.00038476884250614326 tvoygorodpskov.ru
0.00038476884250614326 newvz.ru
0.00038476884250614326 seonews.ru
0.00038476884250614326 vestikavkaza.ru
0.00038476884250614326 overclockers.ua
0.00038476884250614326 news.drom.ru
0.00038476884250614326 rodgor-vlg.ru
0.00038476884250614326 rusjev.net
0.00038476884250614326 fraza.ua
0.00038476884250614326 rusnext.ru
0.00038476884250614326 fergana.agency
0.00038476884250614326 military.pravda.ru
0.00038476884250614326 oreltimes.ru
0.00038476884250614326 livesport.ru
0.00038476884250614326 liganews.net
0.00038476884250614326 t-l.ru
0.00038476884250614326 polit.ru
0.00038476884250614326 newsfrol.ru
0.00038476884250614326 irk.today
0.00038476884250614326 reports.d3.ru
0.00038476884250614326 fognews.d3.ru
0.00038476884250614326 mk-yamal.ru
0.00038476884250614326 pluggedin.ru
0.00038476884250614326 hochu.ua
0.00038476884250614326 allsport-news.net
0.00038476884250614326 infoindustria.com.ua
0.00038476884250614326 progorod62.ru
0.00038476884250614326 vesti-mglin.ru
0.00038476884250614326 news2.ru
0.00038476884250614326 spark.ru
0.00038476884250614326 taminfo.ru
0.00038476884250614326 holodilnik.d3.ru
0.00038476884250614326 batenka.ru
0.00038476884250614326 movies.d3.ru
0.00038476884250614326 weirdrussia.d3.ru
0.00038476884250614326 crime.d3.ru
0.00038476884250614326 hrenovosti.d3.ru
0.00038476884250614326 historyscan.d3.ru
0.00038476884250614326 etcetera.media
0.00038476884250614326 tverigrad.ru
0.00038476884250614326 bel.ru
0.00038476884250614326 20khvylyn.com
0.00038476884250614326 nag.ru
0.00038476884250614326 bankir.ru
0.00038476884250614326 oligarh.media
0.00038476884250614326 vz.ua
0.00038476884250614326 politsovet.ru
0.00038476884250614326 hubs.ua
0.00038476884250614326 organicwoman.ru
0.00038476884250614326 tvkrasnodar.ru
0.00038476884250614326 citydog.by
0.00038476884250614326 dg-yug.ru
0.00038476884250614326 searchengines.ru
0.00038476884250614326 mirmol.ru
0.00038476884250614326 hi-tech.ua
0.00038476884250614326 zeml-trub.ru
0.00038476884250614326 keddr.com
0.00038476884250614326 glasnarod.ru
0.00038476884250614326 tatcenter.ru
0.00038476884250614326 fomag.ru
0.00038476884250614326 dom.vse42.ru
0.00038476884250614326 stapravda.ru
0.00038476884250614326 gmbox.ru
0.00038476884250614326 disgustingmen.com
0.00038476884250614326 borvedomosti.ru
0.00038476884250614326 newsdelo.com
0.00038476884250614326 wylsa.com
0.00038476884250614326 1nsk.ru
0.00038476884250614326 telesputnik.ru
0.00038476884250614326 vostok.today
0.00038476884250614326 3dtoday.ru
0.00038476884250614326 up74.ru
0.00038476884250614326 journal.tinkoff.ru
0.00038476884250614326 trunovskiy.ru
0.00038476884250614326 gazeta-chertanovo-juzhnoe.ru
0.00038476884250614326 itndaily.ru
0.00038476884250614326 ryb.ru
0.00038476884250614326 plitkar.com.ua
0.00038476884250614326 dataisbeautiful.d3.ru
0.00038476884250614326 topnews.ru